# Create an interface between two materials with minimal strain

Use Zur and McGill superlattices matching [algorithm](https://doi.org/10.1063/1.3330840) to create interfaces between two materials using the Pymatgen [implementation](https://pymatgen.org/pymatgen.analysis.interfaces.html#pymatgen.analysis.interfaces.zsl).

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials
2. Set Interface Parameters (e.g. `distance_z`, `max_area`, `miller_indices`) below or use the default values
3. Click "Run" > "Run All Cells" to run all cells
4. Wait for the run to complete (depending on the area, it can take 1-2 min or more). Scroll down to view cell results.
5. Review the strain plot and modify its parameters as needed

## Methodology

The following happens in the script below:

1. Create slabs for each input material. The materials data is passed in from and back to the web application according to this description (TBA).
   We assume that two input materials are either in bulk form (e.g. Ni crystal) or layered (e.g. graphene). 
   
   We construct the interface along the Z-axis. The material corresponding to the bottom of the interface is referred to as the "**substrate**", and the top - as the "**layer**". 

2. Perform strain matching on the slabs to extract the supercell dimensions. The algorithm has a set of parameters, such as the maximum area considered, that can be configured by editing the cells below.

3. When the strain matching is finished, the interface with the lowest strain (and the smallest number of atoms) is selected. We create the corresponding supercells and place them at a specified distance from each other (note no shift is performed currently).


## 1. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` as directed in README.

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb", "../../config.yml")

## 2. Set Interface Parameters

### 2.1. Set Substrate and Layer Parameters 
Imported `InterfaceSettings` is a class that specifies the parameters for the construction of the interface. The default values are assumed if properties are not set during the initialization.
Additionally, specify if the termination selection is done using interactive prompt, or the via selecting the termination index in the code.

In [ ]:
from mat3ra.made.tools.build.interface import InterfaceSettings

# Parameters can be set during the class initialization:
interface_builder_settings = InterfaceSettings(
    distance_z=3.0,                  # distance between two planes, in Angstroms
    use_conventional_cell=True,      # if True, the surface plane is constructed using miller indices of the conventional cell
)

# Parameters can be set after the initialization:
interface_builder_settings.SubstrateParameters.miller_indices = (1, 1, 1) # the Miller indices of the interfacial plane of the substrate
interface_builder_settings.SubstrateParameters.thickness = 3 # substrate thickness in layers
interface_builder_settings.LayerParameters.miller_indices = (0, 0, 1)  # the Miller indices of the interfacial plane of the layer
interface_builder_settings.LayerParameters.thickness = 1 # layer thickness in layers

IS_TERMINATION_SELECTION_INTERACTIVE = False  # if True, the user can select the termination interactively
TERMINATION_INDEX = 0  # the default termination index that is used if no termination selected, ignored in interactive mode

### 2.2. Set Strain Matching Algorithm Parameters (Optional)
The search algorithm for supercells matching can be tuned by setting its parameters directly, otherwise the default values are used.

In [ ]:
from mat3ra.made.tools.build.interface import ZSLParameters
interface_builder_settings.ZSLParameters = ZSLParameters(
    max_area=50,       # maximum area of the generated interfaces, in Angstroms^2
    max_area_tol=0.09,  # maximum tolerance on ratio of super-lattices to consider equal
    max_length_tol=0.03,  # maximum length tolerance for two vectors to be considered equal
    max_angle_tol=0.01, # maximum angle tolerance for two sets of vectors to have equal angles
)

## 3. Load input Materials


In [ ]:
from mat3ra.made.material import Material
from utils.jupyterlite import get_data

from utils.visualize import visualize_materials as visualize

# Get the list of input materials and load them into `materials_in` variable
get_data("materials_in", globals())
materials = list(map(Material, globals()["materials_in"]))
visualize(materials, repetitions=[1, 1, 1], rotation="0x")

## 4. Create interfaces

### 4.1. Initialize the interface builder

Initialize the interface builder with the materials and interface settings.

In [ ]:
from mat3ra.made.tools.build import init_interface_builder

interface_builder = init_interface_builder(
    substrate=materials[0],
    layer=materials[1],
    settings=interface_builder_settings
)

### 4.2. Select the termination
Possible terminations for the interface are found by the interface builder. The user can select the termination interactively or use the default one.

In [ ]:
from utils.io import ui_prompt_select_array_element_by_index, ui_prompt_select_array_element_by_index_pyodide
terminations = interface_builder.terminations

if IS_TERMINATION_SELECTION_INTERACTIVE:
    if sys.platform == "emscripten":
        selected_termination = await ui_prompt_select_array_element_by_index_pyodide(terminations, element_name="termination")
    else:
         selected_termination = ui_prompt_select_array_element_by_index(terminations, element_name="termination")
else:    
    selected_termination = terminations[TERMINATION_INDEX]

### 4.3. Create interfaces for the selected termination

In [ ]:
from mat3ra.made.tools.build import create_interfaces

interface_data_holder = create_interfaces(
    sort_by_strain_and_size=True,
    remove_duplicates=True,
    interface_builder=interface_builder,
    termination=selected_termination,
)

### 4.4. Print out interface with the lowest strain for selected termination

In [ ]:
print(f"Interface with lowest strain for termination {selected_termination} (index 0):")
interfaces = interface_data_holder.get_interfaces_for_termination(selected_termination)
first_interface = interfaces[0]
print(f"    strain: {first_interface.get_mean_abs_strain() * 100:.3f}%")
print("    number of atoms:", first_interface.num_sites)

## 5. Plot the results

Plot the number of atoms vs strain. Adjust the parameters as needed.


In [ ]:
from utils.plot import plot_strain_vs_atoms

PLOT_SETTINGS = {
    "HEIGHT": 600,
    "X_SCALE": "log",  # or linear
    "Y_SCALE": "log",  # or linear
}

plot_strain_vs_atoms(interface_data_holder, PLOT_SETTINGS)

print("Terminations: \n", interface_data_holder.terminations)

## 6. Select the interface to pass outside this kernel

### 6.1. Select the interface with the desired termination and strain

Select the index for termination first, and for it - the index in the list of corresponding interfaces sorted by strain (index 0 has minimum strain).

In [ ]:
# Could be either the termination as tuple, e.g. `('Ni_P6/mmm_1', 'C_C2/m_2')` or its index: `0`
termination_or_its_index = selected_termination
# select the first interface with the lowest strain and the smallest number of atoms
interfaces_slice_range_or_index = slice(0, 1)
selected_interfaces = interface_data_holder.get_interfaces_as_materials(termination_or_its_index, interfaces_slice_range_or_index)

### 6.2. Visualize the selected interface(s)

In [ ]:
visualize(selected_interfaces, repetitions=[1, 1, 1], rotation="0x")

### 6.3. Pass data to the outside runtime
Enrich the selected interfaces names with the strain values and pass them to the application runtime.

In [ ]:
from utils.jupyterlite import set_data

for interface in selected_interfaces:
    if "Interface, Strain:" not in interface["name"]:
        interface["name"] = f'{interface["name"]}, Interface, Strain:{interface["metadata"]["interface_properties"]["mean_abs_strain"]*100:.3f}%'

set_data("materials", selected_interfaces)